# Raman calculation

This notebook presents how to compute a Raman spectrum by using the PyBigDFT API, and more precisely by using the SystemCalculator class.

The goal of these calculations is to find the normal modes of vibration of the system under consideration (their intensities could also be calculated, requiring more calculations). A good reference for the underlying theory of molecular vibrations is the book *Molecular Vibrations* by Wilson *et al.* (1955) or the Advances in Chemical Physics, vol. 67 by K.P. Lawley (where the conversion between atomic units, SI units and the units used in the litterature is well explained).

We will consider the cases of the nitrogen molecule N$_2$

Note: it uses stuff that are not (yet?) in PyBigDFT to manipulate the posinp files.

In [14]:
# Import the useful classes
import os
import numpy as np
from BigDFT.Calculators import SystemCalculator
from BigDFT.Logfiles import Logfile
from inputfiles import Posinp

In [2]:
# Initialize the calculator
study = SystemCalculator(4, 1)

## Equilibrium geometry

**Before running a Raman spectrum calulation, you must have performed a geometry optimization to obtain the reference equilibrium positions of that system.**

However, the goal of this notebook is not to perform a geometry optimization using PyBigDFT, so the posinp file obtained after a precise geometry optimization (using `forcemax: 1.E-6`) is reproduced here as a string:

In [3]:
# Initialize the reference position
N2_ref = """2  angstroem
free
N    2.976307744763e-23    6.872205902435e-23    1.071620018790e-02
N   -1.104344885754e-23   -4.873421785298e-23    1.104273795769e+00"""
pos = Posinp.from_string(N2_ref)

## Run the reference calculation

This is just the calculation of the reference position.

In [4]:
new_dir = "N2_new"
try:
    os.mkdir(new_dir)
except OSError:
    pass
os.chdir(new_dir)
base_name = "N2_ref"
pos.write(base_name+".xyz")

In [5]:
force_run = False  # If True, force the calculation to run
log_exists = os.path.exists("log-{}.yaml".format(base_name))
if (log_exists and force_run) or not log_exists:
    study.run(name="N2_ref")
else:
    print("No need to run the calculation.")

Executing command:  mpirun -np 1 $BIGDFT_ROOT/bigdft -n N2_ref


## Run 6 $n_{at}$ calculations to get the virbational frequencies

To get the **energies** of the Raman spectrum, one needs to find the **eigenvalues of the dynamical matrix**, that is closely related to the Hessian. To build these matrices, one must find the derivatives of the forces when each coordinate of each atom is translated by a small amount around the equilibrium positions. To get a better precision on the derivative, each coodinate is translated positively and negatively, so that the number of BigDFT calculations amounts to $2 * 3 n_{at}$, where $n_{at}$ is the number of atoms (3 for the coordinates ($x$, $y$ and $z$), 2 for the number of calculations per coordinates).

### Initialize $6 n_{at}$ the atom moves beforehand

In [6]:
def get_displacements(amplitudes):
    """
    Function used to define the six similar displacements
    each atom must undergo from the amplitudes of
    displacement in each direction.
    """
    assert len(amplitudes) == 3 
    # Space coordinates
    COORDS = ["x", "y", "z"]
    # Dictionary to convert the string of the signs to floats
    SIGNS = {"+": 1., "-": -1.}
    displacements = []
    for i in range(3):
        for sign in SIGNS:
            name = COORDS[i]+sign
            vector = [0.0]*3
            vector[i] = SIGNS[sign] * amplitudes[i]
            displacements.append({'name': name, 'vector': vector})
    return displacements

def get_moves(ref_posinp, amplitudes):
    """
    Function used to define the posinp file and the name of
    the output directory for each of the 6*n_at atom moves.
    """
    B_TO_ANG = 0.529177249  # Conversion factor from bohr to angstroem
    moves = []
    displacements = get_displacements(amplitudes)
    for i_at, atom in enumerate(pos.atoms):
        # Loop over all the vectors of displacement
        for d in displacements:
            # Set the output directory
            outdir = "atom_{:03d}_{}".format(i_at, d['name'])
            # Set the new posinp.
            # The displacement vector must use the correct units
            vector = d['vector']
            if pos.units == 'angstroem':
                vector = [val*B_TO_ANG for val in vector]
            new_pos = ref_posinp.translate_atom(i_at, vector)
            moves.append({'posinp': new_pos, 'outdir': outdir})
    return moves

# Amplitude of the atom translations along the three coordinates
amplitudes = [0.45/64]*3
moves = get_moves(pos, amplitudes)
assert len(moves) == 6 * pos.n_at

### Run all the moves defined above for each atom

The calculations are run in specific output directories: one for each atom move in a particular direction (there are $6 N_{at}$ = 12 subdirectories).

In [7]:
# If True, force a particular calculation,
# even if a logfile already exists
force_run = False
# Base name of the posinp files
base_name = "N2"
# Loop over all the moves
for move in moves:
    # Move to the output directory
    outdir = move['outdir']
    try:
        os.mkdir(outdir)
    except OSError:
        pass
    os.chdir(outdir)
    # Run the calculation thanks to the calculator
    try:
        # Write the posinp file.
        move['posinp'].write(base_name+".xyz")
        # Run the calculation if desired
        log_exists = os.path.exists("log-{}.yaml".format(base_name))
        if (log_exists and force_run) or not log_exists:
            study.run(name=base_name)
        else:
            print("No need to run the calculation.")
    finally:
        # Make sure to go one directory back, no
        # matter what happens in the try block.
        os.chdir("..")

Executing command:  mpirun -np 1 $BIGDFT_ROOT/bigdft -n N2
Executing command:  mpirun -np 1 $BIGDFT_ROOT/bigdft -n N2
Executing command:  mpirun -np 1 $BIGDFT_ROOT/bigdft -n N2
Executing command:  mpirun -np 1 $BIGDFT_ROOT/bigdft -n N2
Executing command:  mpirun -np 1 $BIGDFT_ROOT/bigdft -n N2
Executing command:  mpirun -np 1 $BIGDFT_ROOT/bigdft -n N2
Executing command:  mpirun -np 1 $BIGDFT_ROOT/bigdft -n N2
Executing command:  mpirun -np 1 $BIGDFT_ROOT/bigdft -n N2
Executing command:  mpirun -np 1 $BIGDFT_ROOT/bigdft -n N2
Executing command:  mpirun -np 1 $BIGDFT_ROOT/bigdft -n N2
Executing command:  mpirun -np 1 $BIGDFT_ROOT/bigdft -n N2
Executing command:  mpirun -np 1 $BIGDFT_ROOT/bigdft -n N2


## Extract vibrational frequencies from these calculations

All these calculations can then be used to build the dynamical matrix, whose eigenvalues are the vibrational frequencies. Some functions are defined in order to do that in a more general case.

In [19]:
def solve_dynamical_matrix(dyn_mat):
    r"""
    Method solving the dynamical matrix to get the phonon energies
    (converted in Hartree) and the eigenvectors.

    :returns: Tuple made of the eigenvalues (as an array) and the
              eigenvectors (as a matrix).
    :rtype: tuple
    """
    eigs, vecs = np.linalg.eig(dyn_mat)
    eigs = np.array([np.sqrt(-e) if e < 0 else np.sqrt(e) for e in eigs])
    return eigs, vecs

def build_dyn_mat(moves, amplitudes, base_name=None):
    r"""
    Method computing the dynamical matrix of the system. It is
    very similar to the Hessian matrix: its elements are only
    corrected by a weight w, which is the inverse of the sqrt of
    the product of the atomic masses of the atoms involved in the
    Hessian matrix element H[i][j]:

    w[i][j] = 1 / \sqrt(mass_i * mass_j)

    where mass_i is is the mass of the atom indexed by i (running
    from 1 to the number of atoms n_at).

    The masses are counted in electronic mass units (which is the
    atomic unit of mass, that is different from the atomic mass
    unit).

    :returns: Dynamical matrix
    :rtype: 2D square np.array of dimension 3*n_at
    """
    # Numpy does the ratio of arrays intellegently: by making
    # masses an array of the same size as the Hessian, there is
    # nothing but the ratio of both arrays to perform to get
    # the dynamical matrix.
    h = build_hessian(moves, amplitudes, base_name)
    masses = build_masses(moves)
    return h / masses

def build_masses(moves):
    r"""
    Method computing the masses array used to define the dynamical
    matrix. The masses are counted in electronic mass units (which
    is the atomic unit of mass, that is different from the atomic
    mass unit).

    :returns: Masses matrix
    :rtype: 2D square np.array of dimension 3*n_at
    """
    # Mass of the different types of atoms in atomic mass units
    # TODO: Add more types of atoms
    #       (found in $SRC_DIR/bigdft/src/orbitals/eleconf-inc.f90)
    MASS_ATOMS = {"H": 1.00794, "He": 4.002602, "Li": 6.941, "Be": 9.012182,
                  "B": 10.811, "C": 12.011, "N": 14.00674, "O": 15.9994,
                  "F": 18.9984032, "Ne": 20.1797}
    # Conversion from atomic to electronic mass unit
    AMU_TO_EMU = 1.660538782e-27 / 9.10938215e-31
    # Get the atoms of the system from the reference posinp
    atoms = moves[0]['posinp'].atoms
    # Build the masses matrix (the loops over range(3) are here
    # to ensure that masses has the same dimension as the Hessian)
    masses = [[np.sqrt(MASS_ATOMS[atom1["Type"]] *
                       MASS_ATOMS[atom2["Type"]])
               for atom2 in atoms for j in range(3)]
              for atom1 in atoms for i in range(3)]
    # Return the masses as a numpy array, converted in electronic
    # mass units
    return np.array(masses)*AMU_TO_EMU

def build_hessian(moves, amplitudes, base_name):
    r"""
    Method computing the Hessian of the system. Its size is 3*n_at
    by 3*n_at, where n_at is the number of atoms of the system.

    :returns: Hessian matrix
    :rtype: 2D square np.array of dimension 3*n_at
    """  # noqa
    # Initialization of variables
    h = []  # Hessian matrix
    n_at = moves[0]['posinp'].n_at  # Number of atoms
    if base_name is not None:
        log_name = "log-{}.yaml".format(base_name)
    else:
        log_name = "log.yaml"
    # First loop over all atoms
    for i_at in range(n_at):
        # Loop over the coordinates (x, y and z)
        for i_coord in range(3):
            # Read the two logfiles corresponding to the moves
            # along the same direction (+ and -)
            j_p = i_at*6 + i_coord*2
            files = [os.path.join(moves[j]['outdir'], log_name)
                     for j in [j_p, j_p+1]]
            logs = [Logfile(filename) for filename in files]
            # The Hessian is made of the delta of the forces
            # with respect to the delta of the move distances.
            # It is built line by line:
            new_line = []
            # 1- Find the delta displacement. It is twice the
            #    distance of the positive move along the direction
            #    of the displacement.
            delta_x = 2 * amplitudes[i_coord]
            # 2- Find the delta forces for each atom and update
            #    the new line of the Hessian.
            for j_at in range(n_at):
                forces = [np.array(log.forces[j_at].values()[0])
                          for log in logs]
                delta_forces = forces[0] - forces[1]
                new_line += list(delta_forces/delta_x)
            # The new line of the Hessian is now complete
            h.append(new_line)
    # Return the Hessian matrix as a numpy array
    return np.array(h)

# Post-processing:
# - Set the dynamical matrix
dyn_mat = build_dyn_mat(moves, amplitudes, base_name=base_name)
# - Find the energies as eigenvalues of the dynamical matrix
# Conversion factor from Hartree to cm^-1
HA_TO_CMM1 = 219474.6313705
energies = {}
energies['Ha'], normal_modes = solve_dynamical_matrix(dyn_mat)
energies['cm^-1'] = energies['Ha'] * HA_TO_CMM1
print(energies['cm^-1'])

[  2.55502098e-05   2.38632021e+03   2.01915188e+01   2.01915188e+01
   6.49312916e-09   1.20474245e-05]


The nitrogen molecule being linear, it has $3 n_{at} - 5 = 1$ normal mode. Its energy, found using the LDA exchange-correlation is around 2386 cm$^{-1}$. Following the statement of [B. G. Johnson *et al.*, *J. Chem. Phys.* **98**, 5612 (1993)](http://aip.scitation.org/doi/10.1063/1.464906), our result should be compared with the harmonic experimental value, which is reported to be 2360 cm$^{-1}$ in this same reference. There is a very good agreement! Note that the actual experimental value (without neglecting anharmocity) is 2331 cm$^{-1}$.